In [18]:
import illustris_python as il
import numpy as np
import matplotlib.pyplot as plt
plt.rcParams["font.family"] = "serif"
plt.rcParams["font.serif"] = "DejaVu Serif"
import matplotlib as mpl
import h5py
import multiprocessing
from tqdm import tqdm



# %matplotlib widget
basePath = "./sims.TNG/TNG50-1/output"
from tqdm import tqdm
import gc
import os

Snapshot_Init = 50
Snapshot_End = 99

h = 0.6774  # km/Mpc

m_dm = 4.5e-5  # E10M_Sum
m_dm_h = m_dm * h

Critial_Density = 1.27e-8  # E10M_Sum/kpc^3
Critial_Density_h = Critial_Density / h**2  # E10M_Sum/(kpc/h)^3

Host_Index_At_99 = 329508
Host_Index_At_98 = 329045
Satellite_Index_At_98 = 329051

Full_Snapshots=np.array([50,59,67,72,78,84,91,99])

In [3]:
Subhalos_Fields=['SubhaloMassType','SubhaloPos','SubhaloMass']
Subhalos_Dict=il.groupcat.loadSubhalos(basePath, Snapshot_End, fields=Subhalos_Fields)

In [8]:
Subhalos_Stellar_Mass=Subhalos_Dict['SubhaloMassType'][:,4]/h
stellar_condition=Subhalos_Stellar_Mass>1
Subhalos_Gas_Mass=Subhalos_Dict['SubhaloMassType'][:,0]/h
gas_condition=Subhalos_Gas_Mass>1
Subhalo_Mass=Subhalos_Dict['SubhaloMass']/h
mass_condition=Subhalo_Mass<10000
Subhalos_Indices=np.where(stellar_condition&gas_condition&mass_condition)[0]


In [16]:
Subhalos_Indices.shape

(706,)

In [17]:
Gas_Fields=['Masses','Coordinates','NeutralHydrogenAbundance']
Stars_Fields=['Masses','Coordinates']

In [5]:
def batch_distance_calculation(pos1, pos2, boxsize=None):
    # 计算每个点与目标点的距离
    dxs = np.abs(pos1[:, 0] - pos2[0])
    dys = np.abs(pos1[:, 1] - pos2[1])
    dzs = np.abs(pos1[:, 2] - pos2[2])

    # 考虑周期性边界条件
    if boxsize is not None:
        dxs2 = np.abs(boxsize - dxs)
        dys2 = np.abs(boxsize - dys)
        dzs2 = np.abs(boxsize - dzs)

        # 在周期性边界条件下，选择最短的距离
        dxs = np.minimum(dxs, dxs2)
        dys = np.minimum(dys, dys2)
        dzs = np.minimum(dzs, dzs2)

    # 计算三维空间点之间的欧氏距离
    distances = np.linalg.norm(np.vstack([dxs, dys, dzs]).T, axis=1)

    return distances


def Radius90(Particles_Coordinates,Particles_Masses,Center):
    boxsize=51700
    Particles_Distance=batch_distance_calculation(Particles_Coordinates,Center,boxsize)
    Particles_Distance_Sorted=np.sort(Particles_Distance)
    Particles_Masses_Sorted=Particles_Masses[np.argsort(Particles_Distance)]
    Masses_Cumulative=np.cumsum(Particles_Masses_Sorted)
    Masses_Cumulative_Normalized=Masses_Cumulative/Masses_Cumulative[-1]
    Radius90=Particles_Distance_Sorted[np.where(Masses_Cumulative_Normalized>=0.9)[0][0]]
    Radius50=Particles_Distance_Sorted[np.where(Masses_Cumulative_Normalized>=0.5)[0][0]]
    return Radius90,Radius50

def Radius90_Process_And_Save(Subhalo_Index):
    Subhalo_Dict=il.groupcat.loadSingle(basePath, Snapshot_End, -1,Subhalo_Index)
    Subhalo_Center=Subhalo_Dict['SubhaloPos']/h
    Subhalo_Mass=Subhalo_Dict['SubhaloMass']/h
    if Subhalo_Mass>10000:
        HI_Radius90=-1
        Stars_Radius90=-1
        File=dict()
        File['Subhalo_Index']=Subhalo_Index
        File['Subhalo_MassType']=Subhalo_Dict['SubhaloMassType']/h
        File['HI_Radius90']=HI_Radius90
        File['HI_Radius50']=HI_Radius90
        File['Stars_Radius50']=Stars_Radius90
        File['Stars_Radius90']=Stars_Radius90
        hdf5_filename=str(Subhalo_Index)+'.hdf5'
        hdf5_path=os.path.join('./Radius90',hdf5_filename)
        with h5py.File(hdf5_path,'w') as f:
            for key in File.keys():
                f[key]=File[key]
        return 0

    Gas_Dict=il.snapshot.loadSubhalo(basePath, Snapshot_End, Subhalo_Index,'gas',fields=Gas_Fields)
    if len(Gas_Dict)==1:
        HI_Radius90=0
        Stars_Radius90=0
        File=dict()
        File['Subhalo_Index']=Subhalo_Index
        File['Subhalo_MassType']=Subhalo_Dict['SubhaloMassType']/h
        File['HI_Radius90']=HI_Radius90
        File['HI_Radius50']=HI_Radius90
        File['Stars_Radius50']=Stars_Radius90
        File['Stars_Radius90']=Stars_Radius90
        hdf5_filename=str(Subhalo_Index)+'.hdf5'
        hdf5_path=os.path.join('./Radius90',hdf5_filename)
        with h5py.File(hdf5_path,'w') as f:
            for key in File.keys():
                f[key]=File[key]
        return 0
    Gas_Coordinates=Gas_Dict['Coordinates']/h
    Gas_Masses=Gas_Dict['Masses']/h
    Gas_NeutralHydrogenAbundance=Gas_Dict['NeutralHydrogenAbundance']
    Gas_NeutralHydrogen_Masses=Gas_Masses*Gas_NeutralHydrogenAbundance



    HI_Radius90=Radius90(Gas_Coordinates,Gas_NeutralHydrogen_Masses,Subhalo_Center)
    del Gas_Dict,Gas_Coordinates,Gas_Masses,Gas_NeutralHydrogenAbundance,Gas_NeutralHydrogen_Masses

    Stars_Dict=il.snapshot.loadSubhalo(basePath, Snapshot_End, Subhalo_Index,'stars',fields=Stars_Fields)
    Stars_Coordinates=Stars_Dict['Coordinates']/h
    Stars_Masses=Stars_Dict['Masses']/h
    Stars_Radius90=Radius90(Stars_Coordinates,Stars_Masses,Subhalo_Center)
    del Stars_Dict,Stars_Coordinates,Stars_Masses

    File=dict()
    File['Subhalo_Index']=Subhalo_Index
    File['Subhalo_MassType']=Subhalo_Dict['SubhaloMassType']/h
    File['HI_Radius90']=HI_Radius90[0]
    File['HI_Radius50']=HI_Radius90[1]
    File['Stars_Radius90']=Stars_Radius90[0]
    File['Stars_Radius50']=Stars_Radius90[1]


    hdf5_filename=str(Subhalo_Index)+'.hdf5'
    hdf5_path=os.path.join('./Radius90',hdf5_filename)
    with h5py.File(hdf5_path,'w') as f:
        for key in File.keys():
            f[key]=File[key]
    return 0

In [6]:
def Detect_Mkdir(Path):
    if not (os.path.exists(Path)):
        os.makedirs(Path)
    return Path

In [9]:
hdf5_filepath=Detect_Mkdir('./Radius90')
import multiprocessing

In [25]:
def process_subhaloindex(i,Subhalo_Indices_Chunks):
    Subhalo_Indices=Subhalo_Indices_Chunks[i]
    for Subhalo_Index in tqdm(Subhalo_Indices):
        Radius90_Process_And_Save(Subhalo_Index)

def chunks(lst, chunk_size):
    for i in range(0, len(lst), chunk_size):
        yield lst[i:i + chunk_size]

processes = []
num_processes = 2
Subhalo_Indices = missing_indices
from random import shuffle
shuffle(Subhalo_Indices)

Subhalo_Indices_Chunks = list(chunks(Subhalo_Indices, len(Subhalo_Indices) // num_processes))

for i in range(num_processes):
    process = multiprocessing.Process(target=process_subhaloindex, args=((i,Subhalo_Indices_Chunks)))
    processes.append(process)

# 启动进程
for process in processes:
    process.start()

# 等待所有进程完成
for process in processes:
    process.join()

print("All processes have finished.")

  0%|          | 0/3 [00:00<?, ?it/s]

100%|██████████| 3/3 [02:31<00:00, 50.42s/it]


All processes have finished.


In [26]:

hdf5_File_Path = './Radius90'
hdf5_files = [f for f in os.listdir(hdf5_File_Path) if f.endswith('.hdf5')]
with h5py.File(os.path.join(hdf5_File_Path, hdf5_files[0]), 'r') as file:
    # 获取文件中的 keys
    hdf5_keys = list(file.keys())

In [27]:
calculated_indices = []
for subindex in range(len(hdf5_files)):
    calculated_indices.append(int(hdf5_files[subindex][:-5]))
calculated_indices = np.array(calculated_indices)
missing_indices = []
for subindex in Subhalos_Stellar_Mass_Indices:
    if subindex not in calculated_indices:
        missing_indices.append(subindex)
missing_indices = np.array(missing_indices)
print(missing_indices.shape,Subhalos_Stellar_Mass_Indices.shape)

(3,) (3295,)


In [28]:
for i in missing_indices:
    Radius90_Process_And_Save(i)

: 

: 

how to identify the disk structure in the simulation using the particle distribution?

1. Find a centre for the subhalo, then define multiple orentations for the disk.
2. For each oreintation, calculate the half mass distance for the subalo, centred at the centre defined before.
3. For each oreintation, calculate the half mass distance for a peculiar disk, centred at the centre defined before.
4. Compare the two half mass distances, if they are similar, then the disk is face-on, otherwise it is edge-on. Then you can say the diskstruc